<a href="https://colab.research.google.com/github/AndreassOlsson/drone-anomalydetection/blob/main/AnomalyDetection_UMCD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Unpickle UMCD Dataloader
2. Reuse Spatiotemporal visiontransformer model
  1. Update convolutional decoder
  2. Enlarge hyperparameters from the reused "temporal" mnist model

In [ ]:
import os
import random
import bisect
import tarfile
import pickle
from glob import glob

import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

import torch
import torch.nn as nn # nn.BCELoss
import torch.optim as optim
import torch.nn.functional as F 
import torch.utils.data as data # data.DataLoader, data.Dataset
import torchvision.transforms as transforms # ToTensor
from tqdm import tqdm



np.random.seed(42)
torch.manual_seed(42)

%matplotlib inline
plt.style.use('ggplot')